In [1]:
import Pkg
using Pkg

Pkg.add("Feather")
Pkg.add("DataFrames")

using Feather, DataFrames, Statistics

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `~/WorkSpaces/CCPRO/julia/marginModel/Project.toml`
 [no changes]
   Updating `~/WorkSpaces/CCPRO/julia/marginModel/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/WorkSpaces/CCPRO/julia/marginModel/Project.toml`
 [no changes]
   Updating `~/WorkSpaces/CCPRO/julia/marginModel/Manifest.toml`
 [no changes]


In [2]:
# preparing holdin periods and confidence levels list
HP = 1
HP_STEP = 1
HP_CNT = 2
CONF = 0.90
CONF_STEP =0.01
CONF_CNT = 3
LOOKBACK_PERIOD=1000


CLstart = Float64(CONF)
CLlist = [CLstart]

while size(CLlist,1) < CONF_CNT
    CL = CLstart + CONF_STEP
    append!(CLlist,CL)
    global CLstart = CL
end
println(CLlist)

[0.9, 0.91, 0.92]


In [3]:
HPstart = Int32(HP)
HPlist = [HPstart]

while size(HPlist,1) < HP_CNT
    HP = HPstart + HP_STEP
    append!(HPlist,HP)
    global HPstart = HP
end

println(HPlist)

Int32[1, 2]


In [4]:
#load feather files
curves_df = Feather.read("./retrieved_curves.feather")

,rf_id,rf_dttm,rf_value,rf_node
,String,Int32,Float32,Int64
1,ITA10YZ,20200702,12.108,120
2,ITA10YZ,20200723,0.99043,120
3,ITA10YZ,20200727,0.99579,120
4,ITA10YZ,20200729,10.001,120
5,ITA10YZ,20200730,0.97361,120
6,ITA10YZ,20200804,0.9576,120
7,ITA10YZ,20200805,0.9823,120
8,ITA10YZ,20200806,0.9404,120
9,ITA10YZ,20200807,0.93291,120


In [5]:
positions_df = Feather.read("./positions.feather")

,position_id,iter_id,account_id,instrument_id,qty,ctv,price
,Int32,Int32,Int32,Int32,Int32,Float32,Float32
1,3200,0,33,0,6534,653400.0,100.0
2,3201,0,33,1,5430,570150.0,105.0
3,3202,0,33,2,9451,992355.0,105.0
4,3203,0,33,3,4184,435136.0,104.0
5,3204,0,33,4,7811,773289.0,99.0
6,3205,0,33,5,8421,850521.0,101.0
7,3206,0,33,6,1273,126027.0,99.0
8,3207,0,33,7,7075,693350.0,98.0
9,3208,0,33,8,3376,351104.0,104.0


In [6]:
instrumentRf_df = Feather.read("./instrumentRf.feather")

,instrument_id,rf_id,bucket_flow
,Int32,String,Float32
1,0,POR8YZ,22.0
2,1,EUR4YZ,51.0
3,1,EUROIS6MZ,73.0
4,2,ESGOV18YZ,20.0
5,2,EURESTOIS10YZ,84.0
6,2,IRL15YZ,54.0
7,3,AUT9YZ,49.0
8,4,DEN6MZ,56.0
9,4,EUR4Y6MZ,44.0


# Scenario_unit

In [7]:
function createScenarios(hp_list, lookback_period, curves_df)
    used_nodes = unique(curves_df.rf_node)
    scenarios_df = DataFrame()
    for hp in hp_list
        for node_str in used_nodes
            node = convert(Int64,node_str)

            #create dataframe for the specific node, order by date descending
            node_df = curves_df[curves_df.rf_node .== node, :]
            if size(node_df,1)<=hp
               error("not enought values for the node " * string(node_str))
            end
            sort!(node_df,[:rf_dttm],rev = true)

            #create shifted_prices vector
            shifted_prices = node_df[(hp+1):(size(node_df, 1)),:rf_value]
            
            #create dataframe adding shifted prices column
            filteredNode_df = first(node_df,length(shifted_prices))

            #create price and shifted price columns
            if node < 1
                price = 100 .* ((-filteredNode_df.rf_node) .* filteredNode_df.rf_value ./ 100)
                sh_price = 100 .* ((-filteredNode_df.rf_node) .* shifted_prices ./ 100)
            else
                price = 100 ./ (1 .+ (filteredNode_df.rf_value ./ 100)) .^ filteredNode_df.rf_node
                sh_price = 100 ./ (1 .+ shifted_prices ./ 100) .^ filteredNode_df.rf_node
            end

            #calculate price ratio
            insertcols!(filteredNode_df,5,:sn_value => price ./ sh_price)

            #get last holding period record
            filteredNode_df = first(filteredNode_df,lookback_period)

            #insert holding period column
            insertcols!(filteredNode_df,6,:hp => hp)
            
            append!(scenarios_df,filteredNode_df)
        end
    end
    
	return select(scenarios_df,Not([:rf_value]))
end

createScenarios (generic function with 1 method)

In [8]:
scenarios_df = createScenarios(HPlist,LOOKBACK_PERIOD,curves_df)
print(scenarios_df)

985×5 DataFrame
│ Row │ rf_id   │ rf_dttm  │ rf_node │ sn_value    │ hp    │
│     │ String  │ Int32    │ Int64   │ Float32     │ Int32 │
├─────┼─────────┼──────────┼─────────┼─────────────┼───────┤
│ 1   │ ITA10YZ │ 20200824 │ 120     │ 0.995899    │ 1     │
│ 2   │ ITA10YZ │ 20200823 │ 120     │ 1.0         │ 1     │
│ 3   │ ITA10YZ │ 20200821 │ 120     │ 0.964659    │ 1     │
│ 4   │ ITA10YZ │ 20200820 │ 120     │ 1.0004      │ 1     │
│ 5   │ ITA10YZ │ 20200819 │ 120     │ 1.01432     │ 1     │
│ 6   │ ITA10YZ │ 20200818 │ 120     │ 1.00466     │ 1     │
│ 7   │ ITA10YZ │ 20200817 │ 120     │ 1.06755     │ 1     │
│ 8   │ ITA10YZ │ 20200816 │ 120     │ 1.0         │ 1     │
│ 9   │ ITA10YZ │ 20200814 │ 120     │ 36015.8     │ 1     │
│ 10  │ ITA10YZ │ 20200813 │ 120     │ 2.69503f-5  │ 1     │
│ 11  │ ITA10YZ │ 20200812 │ 120     │ 0.978015    │ 1     │
│ 12  │ ITA10YZ │ 20200811 │ 120     │ 0.973696    │ 1     │
│ 13  │ ITA10YZ │ 20200810 │ 120     │ 0.999008    │ 1     │
│ 14  │ 

│ 138 │ ITA3MZ  │ 20200806 │ 3       │ 1.0002      │ 1     │
│ 139 │ ITA3MZ  │ 20200805 │ 3       │ 1.0002      │ 1     │
│ 140 │ ITA3MZ  │ 20200804 │ 3       │ 1.00029     │ 1     │
│ 141 │ ITA3MZ  │ 20200803 │ 3       │ 0.999842    │ 1     │
│ 142 │ ITA3MZ  │ 20200802 │ 3       │ 1.0         │ 1     │
│ 143 │ ITA3MZ  │ 20200731 │ 3       │ 0.999806    │ 1     │
│ 144 │ ITA3MZ  │ 20200730 │ 3       │ 0.998839    │ 1     │
│ 145 │ ITA3MZ  │ 20200729 │ 3       │ 1.00043     │ 1     │
│ 146 │ ITA3MZ  │ 20200728 │ 3       │ 1.00079     │ 1     │
│ 147 │ ITA3MZ  │ 20200727 │ 3       │ 1.00051     │ 1     │
│ 148 │ ITA3MZ  │ 20200726 │ 3       │ 1.0         │ 1     │
│ 149 │ ITA3MZ  │ 20200724 │ 3       │ 1.00025     │ 1     │
│ 150 │ ITA3MZ  │ 20200723 │ 3       │ 0.999985    │ 1     │
│ 151 │ ITA3MZ  │ 20200722 │ 3       │ 0.99999     │ 1     │
│ 152 │ ITA3MZ  │ 20200721 │ 3       │ 0.999895    │ 1     │
│ 153 │ ITA3MZ  │ 20200720 │ 3       │ 1.00026     │ 1     │
│ 154 │ ITA3MZ  │ 202007

│ 310 │ ITA6MZ  │ 20200816 │ 6       │ 1.0         │ 1     │
│ 311 │ ITA6MZ  │ 20200814 │ 6       │ 1.00041     │ 1     │
│ 312 │ ITA6MZ  │ 20200813 │ 6       │ 0.999779    │ 1     │
│ 313 │ ITA6MZ  │ 20200812 │ 6       │ 0.999946    │ 1     │
│ 314 │ ITA6MZ  │ 20200811 │ 6       │ 0.999588    │ 1     │
│ 315 │ ITA6MZ  │ 20200810 │ 6       │ 0.999719    │ 1     │
│ 316 │ ITA6MZ  │ 20200809 │ 6       │ 1.0         │ 1     │
│ 317 │ ITA6MZ  │ 20200807 │ 6       │ 1.00034     │ 1     │
│ 318 │ ITA6MZ  │ 20200806 │ 6       │ 1.00033     │ 1     │
│ 319 │ ITA6MZ  │ 20200805 │ 6       │ 0.999775    │ 1     │
│ 320 │ ITA6MZ  │ 20200804 │ 6       │ 1.00164     │ 1     │
│ 321 │ ITA6MZ  │ 20200803 │ 6       │ 0.999956    │ 1     │
│ 322 │ ITA6MZ  │ 20200802 │ 6       │ 1.0         │ 1     │
│ 323 │ ITA6MZ  │ 20200731 │ 6       │ 0.999682    │ 1     │
│ 324 │ ITA6MZ  │ 20200730 │ 6       │ 0.999258    │ 1     │
│ 325 │ ITA6MZ  │ 20200729 │ 6       │ 1.00153     │ 1     │
│ 326 │ ITA6MZ  │ 202007

│ 446 │ ITA8YZ  │ 20200813 │ 96      │ 0.952175    │ 1     │
│ 447 │ ITA8YZ  │ 20200812 │ 96      │ 0.98027     │ 1     │
│ 448 │ ITA8YZ  │ 20200811 │ 96      │ 0.971944    │ 1     │
│ 449 │ ITA8YZ  │ 20200810 │ 96      │ 1.00044     │ 1     │
│ 450 │ ITA8YZ  │ 20200809 │ 96      │ 1.0         │ 1     │
│ 451 │ ITA8YZ  │ 20200807 │ 96      │ 1.00177     │ 1     │
│ 452 │ ITA8YZ  │ 20200806 │ 96      │ 1.04658     │ 1     │
│ 453 │ ITA8YZ  │ 20200805 │ 96      │ 0.983814    │ 1     │
│ 454 │ ITA8YZ  │ 20200804 │ 96      │ 1.05706     │ 1     │
│ 455 │ ITA8YZ  │ 20200803 │ 96      │ 1.00401     │ 1     │
│ 456 │ ITA8YZ  │ 20200802 │ 96      │ 1.0         │ 1     │
│ 457 │ ITA8YZ  │ 20200731 │ 96      │ 0.954563    │ 1     │
│ 458 │ ITA8YZ  │ 20200730 │ 96      │ 1.02538     │ 1     │
│ 459 │ ITA8YZ  │ 20200729 │ 96      │ 1.01604     │ 1     │
│ 460 │ ITA8YZ  │ 20200728 │ 96      │ 0.979988    │ 1     │
│ 461 │ ITA8YZ  │ 20200727 │ 96      │ 1.00916     │ 1     │
│ 462 │ ITA8YZ  │ 202007

│ 583 │ ITA2YZ  │ 20200811 │ 24      │ 0.996604    │ 2     │
│ 584 │ ITA2YZ  │ 20200810 │ 24      │ 1.00166     │ 2     │
│ 585 │ ITA2YZ  │ 20200809 │ 24      │ 1.00264     │ 2     │
│ 586 │ ITA2YZ  │ 20200807 │ 24      │ 1.00672     │ 2     │
│ 587 │ ITA2YZ  │ 20200806 │ 24      │ 1.00096     │ 2     │
│ 588 │ ITA2YZ  │ 20200805 │ 24      │ 1.00471     │ 2     │
│ 589 │ ITA2YZ  │ 20200804 │ 24      │ 1.01008     │ 2     │
│ 590 │ ITA2YZ  │ 20200803 │ 24      │ 1.00223     │ 2     │
│ 591 │ ITA2YZ  │ 20200802 │ 24      │ 0.991562    │ 2     │
│ 592 │ ITA2YZ  │ 20200731 │ 24      │ 0.991631    │ 2     │
│ 593 │ ITA2YZ  │ 20200730 │ 24      │ 1.00449     │ 2     │
│ 594 │ ITA2YZ  │ 20200729 │ 24      │ 0.999253    │ 2     │
│ 595 │ ITA2YZ  │ 20200728 │ 24      │ 0.994147    │ 2     │
│ 596 │ ITA2YZ  │ 20200727 │ 24      │ 0.999285    │ 2     │
│ 597 │ ITA2YZ  │ 20200726 │ 24      │ 0.995915    │ 2     │
│ 598 │ ITA2YZ  │ 20200724 │ 24      │ 1.00156     │ 2     │
│ 599 │ ITA2YZ  │ 202007

│ 828 │ ITA6MZ  │ 20200714 │ 6       │ 0.999508    │ 2     │
│ 829 │ ITA6MZ  │ 20200713 │ 6       │ 0.999852    │ 2     │
│ 830 │ ITA6MZ  │ 20200712 │ 6       │ 1.00036     │ 2     │
│ 831 │ ITA6MZ  │ 20200710 │ 6       │ 0.999866    │ 2     │
│ 832 │ ITA6MZ  │ 20200709 │ 6       │ 0.999437    │ 2     │
│ 833 │ ITA6MZ  │ 20200707 │ 6       │ 1.00008     │ 2     │
│ 834 │ ITA6MZ  │ 20200706 │ 6       │ 1.00015     │ 2     │
│ 835 │ ITA6MZ  │ 20200705 │ 6       │ 0.999064    │ 2     │
│ 836 │ ITA6YZ  │ 20200824 │ 72      │ 0.997707    │ 2     │
│ 837 │ ITA6YZ  │ 20200823 │ 72      │ 0.983056    │ 2     │
│ 838 │ ITA6YZ  │ 20200821 │ 72      │ 0.982116    │ 2     │
│ 839 │ ITA6YZ  │ 20200820 │ 72      │ 1.00477     │ 2     │
│ 840 │ ITA6YZ  │ 20200819 │ 72      │ 1.01602     │ 2     │
│ 841 │ ITA6YZ  │ 20200818 │ 72      │ 1.04874     │ 2     │
│ 842 │ ITA6YZ  │ 20200817 │ 72      │ 1.03812     │ 2     │
│ 843 │ ITA6YZ  │ 20200816 │ 72      │ 1.01353     │ 2     │
│ 844 │ ITA6YZ  │ 202008

# Margin_unit

In [9]:
function portfolioAggregation(account_df,instrRf_df)
	#spread positions per risk factor
	accountRf_df = innerjoin(instrRf_df,account_df,on= :instrument_id)
    
    # calculate flow_countervalue per position per risk factor
	insertcols!(accountRf_df,10,:FLOW_CTV => accountRf_df.bucket_flow ./ 100 .* accountRf_df.ctv)
	
	#aggregate flow_countervalue by account and risk factor
	aggrAccountRf_df = combine(groupby(accountRf_df, [:rf_id, :account_id], sort=true, skipmissing=false), names(accountRf_df, Not([:rf_id, :account_id,:instrument_id,:bucket_flow,:position_id, :iter_id,:qty,:ctv,:price])) .=> sum => :flow_ctv)
    
    return aggrAccountRf_df
end

portfolioAggregation (generic function with 1 method)

In [10]:
aggrAccountRf_df = portfolioAggregation(positions_df,instrumentRf_df)
println(aggrAccountRf_df)

374×3 DataFrame
│ Row │ rf_id         │ account_id │ flow_ctv  │
│     │ String        │ Int32      │ Float32   │
├─────┼───────────────┼────────────┼───────────┤
│ 1   │ AUT10YZ       │ 33         │ 8.01325f6 │
│ 2   │ AUT10YZ       │ 45         │ 9.98594f6 │
│ 3   │ AUT20YZ       │ 33         │ 2.394f6   │
│ 4   │ AUT20YZ       │ 45         │ 2.96813f6 │
│ 5   │ AUT3MZ        │ 33         │ 5.13549f7 │
│ 6   │ AUT3MZ        │ 45         │ 4.7337f7  │
│ 7   │ AUT4YZ        │ 33         │ 7.92591f6 │
│ 8   │ AUT4YZ        │ 45         │ 7.45923f6 │
│ 9   │ AUT6MZ        │ 33         │ 9.31062f6 │
│ 10  │ AUT6MZ        │ 45         │ 8.09041f6 │
│ 11  │ AUT6YZ        │ 33         │ 2.09087f7 │
│ 12  │ AUT6YZ        │ 45         │ 1.92594f7 │
│ 13  │ AUT8YZ        │ 33         │ 1.35407f7 │
│ 14  │ AUT8YZ        │ 45         │ 1.18438f7 │
│ 15  │ AUT9YZ        │ 33         │ 1.34884f7 │
│ 16  │ AUT9YZ        │ 45         │ 1.03788f7 │
│ 17  │ BEL20YZ       │ 33         │ 1.21724f7 │
│ 18

│ 169 │ EURESTOIS9YZ  │ 33         │ 2.70898f7 │
│ 170 │ EURESTOIS9YZ  │ 45         │ 1.81871f7 │
│ 171 │ EUROIS15YZ    │ 33         │ 1.3765f7  │
│ 172 │ EUROIS15YZ    │ 45         │ 1.0668f7  │
│ 173 │ EUROIS1MZ     │ 33         │ 1.91031f7 │
│ 174 │ EUROIS1MZ     │ 45         │ 1.77400f7 │
│ 175 │ EUROIS1YZ     │ 33         │ 624656.0  │
│ 176 │ EUROIS1YZ     │ 45         │ 421744.0  │
│ 177 │ EUROIS25YZ    │ 33         │ 2.24756f7 │
│ 178 │ EUROIS25YZ    │ 45         │ 1.71271f7 │
│ 179 │ EUROIS2MZ     │ 33         │ 3.38383f7 │
│ 180 │ EUROIS2MZ     │ 45         │ 3.81053f7 │
│ 181 │ EUROIS2YZ     │ 33         │ 2.16726f7 │
│ 182 │ EUROIS2YZ     │ 45         │ 1.67965f7 │
│ 183 │ EUROIS3MZ     │ 33         │ 2.41902f6 │
│ 184 │ EUROIS3MZ     │ 45         │ 2.67805f6 │
│ 185 │ EUROIS3YZ     │ 33         │ 3.14946f7 │
│ 186 │ EUROIS3YZ     │ 45         │ 3.58379f7 │
│ 187 │ EUROIS40YZ    │ 33         │ 1.98238f7 │
│ 188 │ EUROIS40YZ    │ 45         │ 1.89981f7 │
│ 189 │ EUROIS6MZ   

│ 343 │ POR4YZ        │ 33         │ 2.0343f7  │
│ 344 │ POR4YZ        │ 45         │ 1.31416f7 │
│ 345 │ POR5YZ        │ 33         │ 1.0117f7  │
│ 346 │ POR5YZ        │ 45         │ 9.24728f6 │
│ 347 │ POR6YZ        │ 33         │ 2.46396f7 │
│ 348 │ POR6YZ        │ 45         │ 2.16714f7 │
│ 349 │ POR8YZ        │ 33         │ 6.14546f7 │
│ 350 │ POR8YZ        │ 45         │ 4.61062f7 │
│ 351 │ POR9YZ        │ 33         │ 2.13209f7 │
│ 352 │ POR9YZ        │ 45         │ 2.41828f7 │
│ 353 │ SPA10YZ       │ 33         │ 5.44764f6 │
│ 354 │ SPA10YZ       │ 45         │ 4.44296f6 │
│ 355 │ SPA15YZ       │ 33         │ 1.73286f7 │
│ 356 │ SPA15YZ       │ 45         │ 1.91958f7 │
│ 357 │ SPA20YZ       │ 33         │ 5.2709f6  │
│ 358 │ SPA20YZ       │ 45         │ 6.66814f6 │
│ 359 │ SPA30YZ       │ 33         │ 3.23495f7 │
│ 360 │ SPA30YZ       │ 45         │ 3.17389f7 │
│ 361 │ SPA3MZ        │ 33         │ 3.96487f7 │
│ 362 │ SPA3MZ        │ 45         │ 4.71386f7 │
│ 363 │ SPA3YZ      

In [13]:
function expectedShortfall(scenarios_df, aggrAccountRf_df,CLlist)
    # evaluate account cash flows for each risk factor
	accountScenario_df = innerjoin(scenarios_df,aggrAccountRf_df, on = :rf_id)
	insertcols!(accountScenario_df,8, :rf_pnl => accountScenario_df.sn_value .* accountScenario_df.flow_ctv .- accountScenario_df.flow_ctv)
    
    # aggregate account cash flows for each scenario
	accountScenario_df = combine(groupby(accountScenario_df, [:account_id,:hp, :rf_dttm], sort=false, skipmissing=false), names(accountScenario_df, ([:rf_pnl])) .=> sum => :account_pnl)
    
    #iterate on subgroups for each account_id and hp
	gdf = groupby(accountScenario_df, [:account_id,:hp])
	exp_short_df = DataFrame()
	
    for g in gdf
        #sort from positive to negative
		sortedG_df = DataFrame(g)
		sort!(sortedG_df,[:account_pnl],rev=true)
    
        #calculate group size
		num_rows=size(sortedG_df,1)
		
		for c in CLlist
            #calculate and extract values in the gaussian tail
			c_val=convert(Int32,round((1 - c) * num_rows,digits=0))
			tailSortedG_Df = last(sortedG_df,c_val)
            
            #calculate expected shortfall and limit
			exp_short_row_df = combine(groupby(tailSortedG_Df, [:account_id,:hp], sort=false, skipmissing=false), names(tailSortedG_Df, :account_pnl) .=> mean => :exp_short, names(tailSortedG_Df, :account_pnl) .=> maximum => :exp_short_lim)
			#note the conficence level into data frame 
            insertcols!(exp_short_row_df,3,:conf => c)
            
			#expected shortfall are negative, convert to positive
            exp_short_row_df.exp_short = abs.(exp_short_row_df.exp_short)
            
            #add to the result data frame
			append!(exp_short_df,exp_short_row_df)
		end
	end
	return exp_short_df
end

expectedShortfall (generic function with 1 method)

In [14]:
expectedShortfall(scenarios_df,aggrAccountRf_df,CLlist)

,account_id,hp,conf,exp_short,exp_short_lim
,Int32,Int32,Float64,Float32,Float32
1,33,1,0.9,1.34872f7,-3.46448f6
2,33,1,0.91,1.34872f7,-3.46448f6
3,33,1,0.92,1.34872f7,-3.46448f6
4,45,1,0.9,1.30301f7,-3.09664f6
5,45,1,0.91,1.30301f7,-3.09664f6
6,45,1,0.92,1.30301f7,-3.09664f6
7,33,2,0.9,1.30505f7,-7.78404f6
8,33,2,0.91,1.30505f7,-7.78404f6
9,33,2,0.92,1.30505f7,-7.78404f6
